In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, IsolationForest
import keras
import matplotlib.pyplot as plt
import tensorflow as tf
import joblib

In [3]:
import pandas as pd
data = pd.read_csv('/home/sid/Documents/6610/csv/web_attacks_balanced.csv')

print(data.head())
print(data.tail())


   Flow ID  Source IP  Source Port  Destination IP  Destination Port  \
0    62015       1261      51885.0            1599              53.0   
1    58525       1261      16641.0            1599              53.0   
2    33764       1256      15954.0            1599              53.0   
3    33749       1256      15744.0            1599              53.0   
4    34500       1256      28467.0            1599              53.0   

   Protocol  Timestamp  Flow Duration  Total Fwd Packets  \
0      17.0        181        76978.0                2.0   
1      17.0        181        78120.0                2.0   
2      17.0        181          205.0                2.0   
3      17.0        181          169.0                2.0   
4      17.0        181          297.0                2.0   

   Total Backward Packets  ...  min_seg_size_forward  Active Mean  Active Std  \
0                     2.0  ...                  32.0          0.0         0.0   
1                     2.0  ...              

In [4]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Check for missing values
print(data.isnull().sum())

# Drop rows with missing values
data = data.dropna()

# Extract features and labels
features = [
    'Protocol', 'Total Length of Fwd Packets', 'Flow Duration', 
    'Total Fwd Packets', 'Total Backward Packets', 'Fwd Packet Length Max', 
    'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 
    'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 
    'Bwd Packet Length Std', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 
    'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 
    'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 
    'Bwd IAT Min'
]
X = data[features]
y = data['Label']

# Encode the 'Protocol' column using one-hot encoding
X_encoded = pd.get_dummies(X, columns=['Protocol'])

# Scale the features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_encoded)


Flow ID             0
Source IP           0
Source Port         0
Destination IP      0
Destination Port    0
                   ..
Idle Mean           0
Idle Std            0
Idle Max            0
Idle Min            0
Label               0
Length: 84, dtype: int64


In [5]:
import tensorflow as tf

# Define autoencoder model
input_dim = X_scaled.shape[1]
encoding_dim = 10

autoencoder = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(input_dim,)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(encoding_dim, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(input_dim, activation='sigmoid')
])

autoencoder.compile(optimizer='adam', loss='mean_squared_error')
autoencoder.fit(X_scaled, X_scaled, epochs=30, batch_size=32)
autoencoder.save('autoencoder_model.h5')

# Use the trained autoencoder for anomaly detection
reconstructed = autoencoder.predict(X_scaled)
mse = np.mean(np.power(X_scaled - reconstructed, 2), axis=1)
threshold = np.percentile(mse, 95)  # Adjust threshold if needed
anomalypoints_autoencoder = data[mse > threshold]

print("Autoencoder anomaly points:")
print(anomalypoints_autoencoder['Label'].value_counts())


Epoch 1/30


/var/data/python/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


228/228 ━━━━━━━━━━━━━━━━━━━━ 1s 650us/step - loss: 0.1003
Epoch 2/30
228/228 ━━━━━━━━━━━━━━━━━━━━ 0s 526us/step - loss: 0.0097
Epoch 3/30
228/228 ━━━━━━━━━━━━━━━━━━━━ 0s 570us/step - loss: 0.0042
Epoch 4/30
228/228 ━━━━━━━━━━━━━━━━━━━━ 0s 552us/step - loss: 0.0020
Epoch 5/30
228/228 ━━━━━━━━━━━━━━━━━━━━ 0s 558us/step - loss: 0.0016
Epoch 6/30
228/228 ━━━━━━━━━━━━━━━━━━━━ 0s 559us/step - loss: 0.0013
Epoch 7/30
228/228 ━━━━━━━━━━━━━━━━━━━━ 0s 551us/step - loss: 0.0013
Epoch 8/30
228/228 ━━━━━━━━━━━━━━━━━━━━ 0s 553us/step - loss: 0.0012
Epoch 9/30
228/228 ━━━━━━━━━━━━━━━━━━━━ 0s 534us/step - loss: 0.0011 
Epoch 10/30
228/228 ━━━━━━━━━━━━━━━━━━━━ 0s 535us/step - loss: 0.0010 
Epoch 11/30
228/228 ━━━━━━━━━━━━━━━━━━━━ 0s 567us/step - loss: 9.1312e-04
Epoch 12/30
228/228 ━━━━━━━━━━━━━━━━━━━━ 0s 624us/step - loss: 8.7477e-04
Epoch 13/30
228/228 ━━━━━━━━━━━━━━━━━━━━ 0s 550us/step - loss: 9.1779e-04
Epoch 14/30
228/228 ━━━━━━━━━━━━━━━━━━━━ 0s 541us/step - loss: 8.4061e-04
Epoch 15/30
228/228 ━━

228/228 ━━━━━━━━━━━━━━━━━━━━ 0s 524us/step
Autoencoder anomaly points:
Label
BENIGN                        340
Web Attack – XSS               19
Web Attack – Brute Force        4
Web Attack – Sql Injection      1
Name: count, dtype: int64


In [6]:
from sklearn.ensemble import IsolationForest, RandomForestClassifier
import joblib

# Fit and predict using IsolationForest
isolation_forest = IsolationForest(contamination=0.25, random_state=42)  # Adjust contamination if needed
anomaly_scores_if = isolation_forest.fit_predict(X_scaled)

# Fit and predict using RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
labels_rf = np.where(anomaly_scores_if == 1, 1, 0)
random_forest.fit(X_scaled, labels_rf)
anomaly_predictions_rf = random_forest.predict(X_scaled)

# Identify anomalies
anomalies_rf = data[anomaly_predictions_rf == 0]

print(f"RandomForest Anomalies: {anomalies_rf}")

# Save the models
joblib.dump(isolation_forest, 'isolation_forest_model.pkl')
joblib.dump(random_forest, 'random_forest_model.pkl')

# Example of loading the models
#loaded_autoencoder = tf.keras.models.load_model('autoencoder_model.h5')
#loaded_isolation_forest = joblib.load('isolation_forest_model.pkl')
#loaded_random_forest = joblib.load('random_forest_model.pkl')


RandomForest Anomalies:       Flow ID  Source IP  Source Port  Destination IP  Destination Port  \
6       41742       1258      49412.0            1599             135.0   
8       71450       1267      52360.0            1599             389.0   
10      49745       1258      49438.0            1599              88.0   
11      75608       1267      49181.0            1599              88.0   
13      75683       1268       1029.0            1599           49671.0   
...       ...        ...          ...             ...               ...   
7248    87068       3937        443.0            1603           52076.0   
7256    85763       1267      51967.0            2125             443.0   
7261    87598       1268       4632.0            1601           23744.0   
7264    89865       4038          0.0            4913               0.0   
7266    14320       1263      61212.0            1590              53.0   

      Protocol  Timestamp  Flow Duration  Total Fwd Packets  \
6          6

['random_forest_model.pkl']

In [7]:
# Count the number of anomalies detected by the autoencoder
num_autoencoder_anomalies = len(anomalypoints_autoencoder)
print("Number of anomalies detected by autoencoder:", num_autoencoder_anomalies)

# Count the number of anomalies detected by the RandomForest
num_rf_anomalies = len(anomalies_rf)
print("Number of anomalies detected by RandomForest:", num_rf_anomalies)

# Count the number of each label in the dataset
label_counts = data['Label'].value_counts()
print(label_counts)


Number of anomalies detected by autoencoder: 364
Number of anomalies detected by RandomForest: 1817
Label
BENIGN                        5087
Web Attack – Brute Force      1507
Web Attack – XSS               652
Web Attack – Sql Injection      21
Name: count, dtype: int64


In [8]:
# Cross-check actual labels with detected anomalies for RandomForest
anomalies_rf_labels = anomalies_rf['Label'].value_counts()
print("Labels of RandomForest detected anomalies:")
print(anomalies_rf_labels)
print()
anomalypoints_autoencoder_labels = anomalypoints_autoencoder['Label'].value_counts()
print("Labels of autoencoder detected anomalies:")
print(anomalypoints_autoencoder_labels)

Labels of RandomForest detected anomalies:
Label
BENIGN                        1630
Web Attack – Brute Force       151
Web Attack – XSS                24
Web Attack – Sql Injection      12
Name: count, dtype: int64

Labels of autoencoder detected anomalies:
Label
BENIGN                        340
Web Attack – XSS               19
Web Attack – Brute Force        4
Web Attack – Sql Injection      1
Name: count, dtype: int64


In [9]:
from sklearn.metrics import classification_report, confusion_matrix

# Create true labels for comparison
true_labels = y.map({'BENIGN': 0, 'Web Attack – Brute Force': 1, 'Web Attack – XSS': 1, 'Web Attack – Sql Injection': 1})

# For Autoencoder
pred_autoencoder = mse > threshold
print("Autoencoder Classification Report:")
print(classification_report(true_labels, pred_autoencoder))

# For RandomForest
pred_rf = anomaly_predictions_rf == 0
print("RandomForest Classification Report:")
print(classification_report(true_labels, pred_rf))


Autoencoder Classification Report:
              precision    recall  f1-score   support

           0       0.69      0.93      0.79      5087
           1       0.07      0.01      0.02      2180

    accuracy                           0.66      7267
   macro avg       0.38      0.47      0.41      7267
weighted avg       0.50      0.66      0.56      7267

RandomForest Classification Report:
              precision    recall  f1-score   support

           0       0.63      0.68      0.66      5087
           1       0.10      0.09      0.09      2180

    accuracy                           0.50      7267
   macro avg       0.37      0.38      0.37      7267
weighted avg       0.47      0.50      0.49      7267

